In [1]:
import pandas as pd
import numpy as np
import pickle
import json
import sys
sys.path.append('.')

### Load Best Model

In [3]:
ctabgan_best = pickle.load(open('../thesisgan/output/ctabgan_best_model/pklmodel.pkl', 'rb'))

/home/sangtani/.conda/envs/ctabgan_new/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/sangtani/.conda/envs/ctabgan_new/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator BayesianGaussianMixture from version 1.3.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

### Conditional Sampling

In [3]:
train_data = pd.read_csv('../thesisgan/input/new_train_data.csv')
hpo_data = pd.read_csv("../thesisgan/input/new_hpo_data.csv")
test_data = pd.read_csv("../thesisgan/input/new_test_data.csv")

In [4]:
og_syn_data = pd.read_csv("../research/best_models/ctabgan_best_model/syn.csv")

#### Get ML Utility Functions

In [5]:
from model.evaluation import get_utility_metrics
from sklearn.preprocessing import LabelEncoder
from model.data_preparation import DataPrep

In [6]:
data_prep = DataPrep(train_data, ['proto', 'tcp_ack', 'tcp_psh', 'tcp_rst', 'tcp_syn', 'tcp_fin', 'tos', 'label', 'attack_type'], 
                     [], {}, [], ['packets', 'src_pt', 'dst_pt', 'duration', 'bytes'], [],
                     {"Classification": 'attack_type'}, 0.00003)

In [7]:
more_samples = ctabgan_best.sample(train_data.shape[0])
og_more_samples = data_prep.inverse_prep(more_samples)

Sampled 274245 invalid samples. Have 890755 samples. Want 1164332 samples.
Sampled 65101 invalid samples. Have 1100654 samples. Want 1164332 samples.
Time taken: 10.380073070526123 seconds
Sampled 15454 invalid samples. Have 1151200 samples. Want 1164332 samples.
Time taken: 2.449706792831421 seconds
Sampled 3748 invalid samples. Have 1163452 samples. Want 1164332 samples.
Time taken: 0.6998398303985596 seconds
Sampled 881 invalid samples. Have 1166571 samples. Want 1164332 samples.
Time taken: 0.22004318237304688 seconds


### Evaluate the Results

#### Original Results - train data, hpo data, fake data size of hpo data

In [8]:
import json
with open('../research/best_models/ctabgan_best_model/table/Classification Results_305_46aa8d602dd61325aed1.table.json') as f:
    og_cr = json.load(f)

In [9]:
le_dict = {"attack_type": "le_attack_type", "label": "le_label", "proto": "le_proto", "tos": "le_tos"}
for c in le_dict.keys():
    le_dict[c] = LabelEncoder()
    test_data[c] = le_dict[c].fit_transform(test_data[c])
    train_data[c] = le_dict[c].fit_transform(train_data[c])
    hpo_data[c] = le_dict[c].fit_transform(hpo_data[c])
    og_syn_data[c] = le_dict[c].fit_transform(og_syn_data[c])
    og_more_samples[c] = le_dict[c].fit_transform(og_more_samples[c])

In [10]:
og_cr = pd.DataFrame(data=og_cr["data"], columns=og_cr["columns"])

In [11]:
og_cr.drop(["Model"],axis=1).groupby(["Type"]).mean().sort_values(by="F1_Score", ascending=False).head(100)


,Acc,AUC,F1_Score
Type,,,
Real,78.035474,0.891350,0.584898
Fake,52.619821,0.832858,0.495669
Difference,25.415653,0.058492,0.089229


#### Original but more - train data, hpo data, fake data size of train data

In [26]:
more_samples_results, more_samples_cr = get_utility_metrics(train_data, hpo_data, og_more_samples, scaler="MinMax",type={"Classification":["xgb","lr","dt","rf","mlp"]})

Model:  xgb trained on real data
Model:  lr trained on real data
Model:  dt trained on real data
Model:  rf trained on real data
Model:  mlp trained on real data
Model:  xgb trained on fake data
Model:  lr trained on fake data
Model:  dt trained on fake data



## Look at the data of all the results

In [1]:
import pandas as pd
import numpy as np


In [3]:
report = pd.read_csv("../thesisgan/output/ctabgan_best_model/table/ctabgan_best_reports.csv")
results = pd.read_csv("../thesisgan/output/ctabgan_best_model/table/ctabgan_best_results.csv")

In [8]:
results["Desc"].unique()

array(['Original with hpo sized samples (train-hpo (hposized))',
       'Original with train sized samples (train-hpo (trainsized))',
       'Original with hpo sized samples, more test data (train- test+hpo))',
       'Original with train sized samples, more test data (train- test+hpo))'],
      dtype=object)

In [6]:
results

,Acc,AUC,F1_Score,Type,Desc,SE_Acc,SE_AUC,SE_F1
0,78.035474,0.891350,0.584898,Real,Original with hpo sized samples (train-hpo (hp...,NaN,NaN,NaN
1,52.619821,0.832858,0.495669,Fake,Original with hpo sized samples (train-hpo (hp...,NaN,NaN,NaN
2,25.415653,0.058492,0.089229,Difference,Original with hpo sized samples (train-hpo (hp...,NaN,NaN,NaN
3,76.549789,0.887136,0.574203,Real,Original with train sized samples (train-hpo (...,0.072118,0.000527,0.000826
4,52.832470,0.849401,0.453714,Fake,Original with train sized samples (train-hpo (...,0.087015,0.000612,0.000831
5,23.717319,0.037735,0.120488,Difference,Original with train sized samples (train-hpo (...,-0.014897,-0.000085,-0.000005
6,81.974390,0.929670,0.650088,Real,"Original with hpo sized samples, more test dat...",0.046067,0.000303,0.000552
7,53.162439,0.834595,0.503570,Fake,"Original with hpo sized samples, more test dat...",0.061163,0.000436,0.000573
8,28.811952,0.095075,0.146519,Difference,"Original with hpo sized samples, more test dat...",-0.015096,-0.000133,-0.000020
9,81.974390,0.929670,0.650088,Real,"Original with train sized samples, more test d...",0.046067,0.000303,0.000552
